# Organize semi continuous Brunel

Organize data of continuous Brunel model with discrete delays, equal probability of each delay value.

In [41]:
#import nest
import pandas as pd
import elephant
import quantities as pq
import neo
import numpy as np
import sys
sys.setrecursionlimit(10000)
from pathlib import Path
from elephant.spike_train_correlation import correlation_coefficient
import scipy.stats as stats
import itertools
from elephant.conversion import BinnedSpikeTrain
# import sys
# !{sys.executable} -m pip install elephant

Create dataframes of cv distributions all resolutions all seeds.

In [2]:
def get_cvs(spike_data):
    """
    Get the CV for each neuron recorded. 
    
    CV = standard_deviation(ISIs)/mean(ISIs)
    """
    cvs = []
     
    spike_data = spike_data.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])

    for name, group in grouped:
            """
            Each group is senders and times for one value of senders. That is, we iterate through all 
            neurons. And the times for each neuron is in sorted order. Therefore, the cvs
            returned must have the same order. So cvs contain cv of neuron 1, then neuron 2 .... then neuron N.
            """
            t = np.asarray(group['time_ms'])
            spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
            isi = elephant.statistics.isi(spiketrain)
            cv = elephant.statistics.cv(isi)
            cvs.append(cv)
            
    return cvs

In [20]:
def create_cv_dfs():
    path = Path('C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data')
    for p in path.iterdir():
        # get list of all files of excitatory spike times in semi cont brunel in one resolution
        file = list(p.glob('semi_cont_brunel/*12502-0.dat'))
        cv_list = list()
        for f in file:
            df = pd.read_csv(f,skiprows=2,sep='\t')
            cvs = get_cvs(df) # get cvs of one seed value one resolution
            cv_list.append(cvs)
        df =pd.DataFrame(cv_list)
        # save the dataframe as semi_cont_cv.csv in each resolution folder
        df.to_csv(r'{}\semi_cont_cv.csv'.format(p))

In [23]:
create_cv_dfs()

Create pickle files of FR distributions all resolutions all seeds.

In [24]:
def get_frs(spike_data):
    """
    Get the FR for each neuron recorded. 
    
    FR = number of spikes fired during simulation/time of simulation
    """
    frs = []
     
    spike_data = spike_data.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])

    for name, group in grouped:
            """
            Each group is senders and times for one value of senders. That is, we iterate through all 
            neurons. And the times for each neuron is in sorted order. Therefore, the cvs
            returned must have the same order. So cvs contain cv of neuron 1, then neuron 2 .... then neuron N.
            """
            t = np.asarray(group['time_ms'])
            spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
            fr = elephant.statistics.mean_firing_rate(spiketrain)
            frs.append(fr)
            
    return frs

In [27]:
def create_fr_dfs():
    path = Path('C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data')
    for p in path.iterdir():
        # get list of all files of excitatory spike times in semi cont brunel in one resolution
        file = list(p.glob('semi_cont_brunel/*12502-0.dat'))
        fr_list = list()
        for f in file:
            df = pd.read_csv(f,skiprows=2,sep='\t')
            frs = get_frs(df) # get cvs of one seed value one resolution
            fr_list.append(frs)
        df = pd.DataFrame(fr_list).fillna(pq.quantity.Quantity(0,units='1/ms')) # save frs to dataframe
        df.to_pickle(r'{}\semi_cont_fr.pkl'.format(p)) # save as csv file 

In [28]:
create_fr_dfs()

Create pickle files of CC distributions all resolutions all seeds.

In [29]:
def get_first_inh(inh):
    """
    Takes in path to dat file of inhibitory population spike times.
    
    This function retrieves the spike times of the first neuron in the inhibitory population.
    Return it as a spiketrain object.
    """
    #inh = pd.read_csv(path, skiprows=2, sep='\t')
    spike_data = inh.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])
    t = np.array(grouped.get_group(10001)['time_ms'])
    spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
    
    return spiketrain

In [30]:
def get_ccs(exc, first_inh, bs=5):
    """
    Get the CC between each excitatory neuron and first inhibitory neuron.
    CC = (x-mean(x))(y-mean(y))/sqrt((x-mean(x))^2(y-mean(y))^2)
    """
    # get all spike trains of excitatory neurons
    spike_data = exc.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])
    spike_trains = []
    for name, group in grouped:
        t = np.asarray(group['time_ms'])
        spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
        spike_trains.append(spiketrain)
    
    correlations = []
    for i in range(len(spike_trains)):
        # calculate cc matrix for the pair of spiketrains
        cc_matrix = correlation_coefficient(BinnedSpikeTrain([spike_trains[i],first_inh], bin_size=bs*pq.ms))
        # retrieve element not on diagonala
        cc = cc_matrix[1,0]
        correlations.append(cc)
        
    return correlations

In [35]:
def get_all_ccs(exc,inh,bs=5):
    """
    This function takes in two nested lists. exc is list of tables of spike times for excitatory popuulation.
    Same for inh but inhibitory population.
    """
    cc_list = list()
    for i in range(1,11):
        first_inh = get_first_inh(inh[i])
        exc_ccs = get_ccs(exc[i],first_inh,bs=bs)
        cc_list.append(exc_ccs)
    return cc_list

In [38]:
def get_data(spike_path):
    """
    Get spike data from brunel model with rounded spike times and delays drawn from discrete interval [1.0, 2.0].
    Send in any path we need.
    """
    #spike_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\brunel_rounding_1_2'
    exc = {}
    for i in range(1,11):
        exc[i] = pd.read_csv(r'{}/brunel_semi_continuous_seed_{}_spikes_exc-12502-0.dat'.format(spike_path, i),
                             skiprows=2,sep='\t')

    inh = {}
    for i in range(1,11):
        inh[i] = pd.read_csv(r'{}/brunel_semi_continuous__seed_{}_spikes_inh-12503-0.dat'.format(spike_path, i),
                             skiprows=2,sep='\t')
    return exc, inh

In [39]:
spike_paths = ['C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\semi_cont_brunel',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_4\\semi_cont_brunel',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\semi_cont_brunel',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_16\\semi_cont_brunel',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_32\\semi_cont_brunel',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_64\\semi_cont_brunel',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_128\\semi_cont_brunel',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_256\\semi_cont_brunel']

In [42]:
for i in range(8):
    exc, inh = get_data(spike_paths[i])
    cc_list_5 = get_all_ccs(exc,inh,bs=5)
    cc_list_10 = get_all_ccs(exc,inh,bs=10)
    cc_list_20 = get_all_ccs(exc,inh,bs=20)
    cc_list_50 = get_all_ccs(exc,inh,bs=50)
    cc_5 = pd.DataFrame(cc_list_5)
    cc_10 = pd.DataFrame(cc_list_10)
    cc_20 = pd.DataFrame(cc_list_20)
    cc_50 = pd.DataFrame(cc_list_50)
    cc_5.to_pickle('{}\cc_df_5.pkl'.format(spike_paths[i]))
    cc_10.to_pickle('{}\cc_df_10.pkl'.format(spike_paths[i]))
    cc_20.to_pickle('{}\cc_df_20.pkl'.format(spike_paths[i]))
    cc_50.to_pickle('{}\cc_df_50.pkl'.format(spike_paths[i]))

C:\Users\Nida\anaconda3\lib\site-packages\elephant\utils.py:290: UserWarning: Correcting 1 rounding errors by shifting the affected spikes into the following bin. You can set tolerance=None to disable this behaviour.
  warnings.warn(f'Correcting {num_corrections} rounding errors by '


Now create a multiindex dataframe for the semi continuous brunel model alone, can think of adding it to the other df for brunel models if it makes sense.

In [45]:
main_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data'
resolution = ['resolution_1_2','resolution_1_4','resolution_1_8','resolution_1_16','resolution_1_32','resolution_1_64',
      'resolution_1_128','resolution_1_256']
file = 'semi_cont_cv.csv'


cv_1_2 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[0],file),index_col=0)



cv_1_4 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[1],file),index_col=0)


cv_1_8 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[2],file),index_col=0)

cv_1_16 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[3],file),index_col=0)

cv_1_32 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[4],file),index_col=0)

cv_1_64 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[5],file),index_col=0)

cv_1_128 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[6],file),index_col=0)

cv_1_256 = pd.read_csv(r'{}\{}\{}'.format(main_path,resolution[7],file),index_col=0)


In [48]:
main_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data'
resolution = ['resolution_1_2','resolution_1_4','resolution_1_8','resolution_1_16','resolution_1_32','resolution_1_64',
      'resolution_1_128','resolution_1_256']
file = 'semi_cont_fr.pkl'


fr_1_2 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[0],file))


fr_1_4 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[1],file))


fr_1_8 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[2],file))

fr_1_16 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[3],file))

fr_1_32 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[4],file))

fr_1_64 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[5],file))

fr_1_128 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[6],file))

fr_1_256 = pd.read_pickle(r'{}\{}\{}'.format(main_path,resolution[7],file))


In [50]:
main_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data'
res = ['resolution_1_2','resolution_1_4','resolution_1_8','resolution_1_16','resolution_1_32','resolution_1_64',
      'resolution_1_128','resolution_1_256']
files = ['semi_cont_brunel\\cc_df_5.pkl','semi_cont_brunel\\cc_df_10.pkl','semi_cont_brunel\\cc_df_20.pkl',
         'semi_cont_brunel\\cc_df_50.pkl']

cc_1_2_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[0],files[0]))
cc_1_2_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[0],files[1]))
cc_1_2_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[0],files[2]))
cc_1_2_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[0],files[3]))

cc_1_4_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[1],files[0]))
cc_1_4_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[1],files[1]))
cc_1_4_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[1],files[2]))
cc_1_4_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[1],files[3]))

cc_1_8_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[2],files[0]))
cc_1_8_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[2],files[1]))
cc_1_8_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[2],files[2]))
cc_1_8_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[2],files[3]))

cc_1_16_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[3],files[0]))
cc_1_16_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[3],files[1]))
cc_1_16_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[3],files[2]))
cc_1_16_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[3],files[3]))

cc_1_32_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[4],files[0]))
cc_1_32_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[4],files[1]))
cc_1_32_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[4],files[2]))
cc_1_32_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[4],files[3]))

cc_1_64_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[5],files[0]))
cc_1_64_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[5],files[1]))
cc_1_64_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[5],files[2]))
cc_1_64_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[5],files[3]))

cc_1_128_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[6],files[0]))
cc_1_128_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[6],files[1]))
cc_1_128_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[6],files[2]))
cc_1_128_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[6],files[3]))

cc_1_256_5 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[7],files[0]))
cc_1_256_10 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[7],files[1]))
cc_1_256_20 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[7],files[2]))
cc_1_256_50 = pd.read_pickle(r'{}\{}\{}'.format(main_path,res[7],files[3]))


Put all the values into a multiindex dataframe

In [51]:
second = [2,4,8,16,32,64,128,256] # what type of resolution
third = [1,2,3,4,5,6,7,8,9,10] # what seed
fourth = [i for i in range(0,10000)]
arrays = [second, third, fourth]


index = pd.MultiIndex.from_product(arrays)
columns = ['CV','FR','CC_5','CC_10','CC_20','CC_50']

df = pd.DataFrame(index=index, columns=columns)

# indexes must be sorted to make more effective and avoid warnings.
df2 = df.sort_index()

In [53]:
df2.loc[(2,1)]['CV'][:] = cv_1_2.iloc[0]
df2.loc[(2,2)]['CV'][:] = cv_1_2.iloc[1]
df2.loc[(2,3)]['CV'][:] = cv_1_2.iloc[2]
df2.loc[(2,4)]['CV'][:] = cv_1_2.iloc[3]
df2.loc[(2,5)]['CV'][:] = cv_1_2.iloc[4]
df2.loc[(2,6)]['CV'][:] = cv_1_2.iloc[5]
df2.loc[(2,7)]['CV'][:] = cv_1_2.iloc[6]
df2.loc[(2,8)]['CV'][:] = cv_1_2.iloc[7]
df2.loc[(2,9)]['CV'][:] = cv_1_2.iloc[8]
df2.loc[(2,10)]['CV'][:] = cv_1_2.iloc[9]

In [54]:
df2.loc[(4,1)]['CV'][:] = cv_1_4.iloc[0]
df2.loc[(4,2)]['CV'][:] = cv_1_4.iloc[1]
df2.loc[(4,3)]['CV'][:] = cv_1_4.iloc[2]
df2.loc[(4,4)]['CV'][:] = cv_1_4.iloc[3]
df2.loc[(4,5)]['CV'][:] = cv_1_4.iloc[4]
df2.loc[(4,6)]['CV'][:] = cv_1_4.iloc[5]
df2.loc[(4,7)]['CV'][:] = cv_1_4.iloc[6]
df2.loc[(4,8)]['CV'][:] = cv_1_4.iloc[7]
df2.loc[(4,9)]['CV'][:] = cv_1_4.iloc[8]
df2.loc[(4,10)]['CV'][:] = cv_1_4.iloc[9]

In [55]:
df2.loc[(8,1)]['CV'][:] = cv_1_8.iloc[0]
df2.loc[(8,2)]['CV'][:] = cv_1_8.iloc[1]
df2.loc[(8,3)]['CV'][:] = cv_1_8.iloc[2]
df2.loc[(8,4)]['CV'][:] = cv_1_8.iloc[3]
df2.loc[(8,5)]['CV'][:] = cv_1_8.iloc[4]
df2.loc[(8,6)]['CV'][:] = cv_1_8.iloc[5]
df2.loc[(8,7)]['CV'][:] = cv_1_8.iloc[6]
df2.loc[(8,8)]['CV'][:] = cv_1_8.iloc[7]
df2.loc[(8,9)]['CV'][:] = cv_1_8.iloc[8]
df2.loc[(8,10)]['CV'][:] = cv_1_8.iloc[9]

In [56]:
df2.loc[(16,1)]['CV'][:] = cv_1_16.iloc[0]
df2.loc[(16,2)]['CV'][:] = cv_1_16.iloc[1]
df2.loc[(16,3)]['CV'][:] = cv_1_16.iloc[2]
df2.loc[(16,4)]['CV'][:] = cv_1_16.iloc[3]
df2.loc[(16,5)]['CV'][:] = cv_1_16.iloc[4]
df2.loc[(16,6)]['CV'][:] = cv_1_16.iloc[5]
df2.loc[(16,7)]['CV'][:] = cv_1_16.iloc[6]
df2.loc[(16,8)]['CV'][:] = cv_1_16.iloc[7]
df2.loc[(16,9)]['CV'][:] = cv_1_16.iloc[8]
df2.loc[(16,10)]['CV'][:] = cv_1_16.iloc[9]

In [57]:
df2.loc[(32,1)]['CV'][:] = cv_1_32.iloc[0]
df2.loc[(32,2)]['CV'][:] = cv_1_32.iloc[1]
df2.loc[(32,3)]['CV'][:] = cv_1_32.iloc[2]
df2.loc[(32,4)]['CV'][:] = cv_1_32.iloc[3]
df2.loc[(32,5)]['CV'][:] = cv_1_32.iloc[4]
df2.loc[(32,6)]['CV'][:] = cv_1_32.iloc[5]
df2.loc[(32,7)]['CV'][:] = cv_1_32.iloc[6]
df2.loc[(32,8)]['CV'][:] = cv_1_32.iloc[7]
df2.loc[(32,9)]['CV'][:] = cv_1_32.iloc[8]
df2.loc[(32,10)]['CV'][:] = cv_1_32.iloc[9]

In [58]:
df2.loc[(64,1)]['CV'][:] = cv_1_64.iloc[0]
df2.loc[(64,2)]['CV'][:] = cv_1_64.iloc[1]
df2.loc[(64,3)]['CV'][:] = cv_1_64.iloc[2]
df2.loc[(64,4)]['CV'][:] = cv_1_64.iloc[3]
df2.loc[(64,5)]['CV'][:] = cv_1_64.iloc[4]
df2.loc[(64,6)]['CV'][:] = cv_1_64.iloc[5]
df2.loc[(64,7)]['CV'][:] = cv_1_64.iloc[6]
df2.loc[(64,8)]['CV'][:] = cv_1_64.iloc[7]
df2.loc[(64,9)]['CV'][:] = cv_1_64.iloc[8]
df2.loc[(64,10)]['CV'][:] = cv_1_64.iloc[9]

In [59]:
df2.loc[(128,1)]['CV'][:] = cv_1_128.iloc[0]
df2.loc[(128,2)]['CV'][:] = cv_1_128.iloc[1]
df2.loc[(128,3)]['CV'][:] = cv_1_128.iloc[2]
df2.loc[(128,4)]['CV'][:] = cv_1_128.iloc[3]
df2.loc[(128,5)]['CV'][:] = cv_1_128.iloc[4]
df2.loc[(128,6)]['CV'][:] = cv_1_128.iloc[5]
df2.loc[(128,7)]['CV'][:] = cv_1_128.iloc[6]
df2.loc[(128,8)]['CV'][:] = cv_1_128.iloc[7]
df2.loc[(128,9)]['CV'][:] = cv_1_128.iloc[8]
df2.loc[(128,10)]['CV'][:] = cv_1_128.iloc[9]

In [60]:
df2.loc[(256,1)]['CV'][:] = cv_1_256.iloc[0]
df2.loc[(256,2)]['CV'][:] = cv_1_256.iloc[1]
df2.loc[(256,3)]['CV'][:] = cv_1_256.iloc[2]
df2.loc[(256,4)]['CV'][:] = cv_1_256.iloc[3]
df2.loc[(256,5)]['CV'][:] = cv_1_256.iloc[4]
df2.loc[(256,6)]['CV'][:] = cv_1_256.iloc[5]
df2.loc[(256,7)]['CV'][:] = cv_1_256.iloc[6]
df2.loc[(256,8)]['CV'][:] = cv_1_256.iloc[7]
df2.loc[(256,9)]['CV'][:] = cv_1_256.iloc[8]
df2.loc[(256,10)]['CV'][:] = cv_1_256.iloc[9]

now frs

In [61]:
df2.loc[(2,1)]['FR'][:] = fr_1_2.iloc[0]
df2.loc[(2,2)]['FR'][:] = fr_1_2.iloc[1]
df2.loc[(2,3)]['FR'][:] = fr_1_2.iloc[2]
df2.loc[(2,4)]['FR'][:] = fr_1_2.iloc[3]
df2.loc[(2,5)]['FR'][:] = fr_1_2.iloc[4]
df2.loc[(2,6)]['FR'][:] = fr_1_2.iloc[5]
df2.loc[(2,7)]['FR'][:] = fr_1_2.iloc[6]
df2.loc[(2,8)]['FR'][:] = fr_1_2.iloc[7]
df2.loc[(2,9)]['FR'][:] = fr_1_2.iloc[8]
df2.loc[(2,10)]['FR'][:] = fr_1_2.iloc[9]

In [62]:
df2.loc[(4,1)]['FR'][:] = fr_1_4.iloc[0]
df2.loc[(4,2)]['FR'][:] = fr_1_4.iloc[1]
df2.loc[(4,3)]['FR'][:] = fr_1_4.iloc[2]
df2.loc[(4,4)]['FR'][:] = fr_1_4.iloc[3]
df2.loc[(4,5)]['FR'][:] = fr_1_4.iloc[4]
df2.loc[(4,6)]['FR'][:] = fr_1_4.iloc[5]
df2.loc[(4,7)]['FR'][:] = fr_1_4.iloc[6]
df2.loc[(4,8)]['FR'][:] = fr_1_4.iloc[7]
df2.loc[(4,9)]['FR'][:] = fr_1_4.iloc[8]
df2.loc[(4,10)]['FR'][:] = fr_1_4.iloc[9]

In [63]:
df2.loc[(8,1)]['FR'][:] = fr_1_8.iloc[0]
df2.loc[(8,2)]['FR'][:] = fr_1_8.iloc[1]
df2.loc[(8,3)]['FR'][:] = fr_1_8.iloc[2]
df2.loc[(8,4)]['FR'][:] = fr_1_8.iloc[3]
df2.loc[(8,5)]['FR'][:] = fr_1_8.iloc[4]
df2.loc[(8,6)]['FR'][:] = fr_1_8.iloc[5]
df2.loc[(8,7)]['FR'][:] = fr_1_8.iloc[6]
df2.loc[(8,8)]['FR'][:] = fr_1_8.iloc[7]
df2.loc[(8,9)]['FR'][:] = fr_1_8.iloc[8]
df2.loc[(8,10)]['FR'][:] = fr_1_8.iloc[9]

In [64]:
df2.loc[(16,1)]['FR'][:] = fr_1_16.iloc[0]
df2.loc[(16,2)]['FR'][:] = fr_1_16.iloc[1]
df2.loc[(16,3)]['FR'][:] = fr_1_16.iloc[2]
df2.loc[(16,4)]['FR'][:] = fr_1_16.iloc[3]
df2.loc[(16,5)]['FR'][:] = fr_1_16.iloc[4]
df2.loc[(16,6)]['FR'][:] = fr_1_16.iloc[5]
df2.loc[(16,7)]['FR'][:] = fr_1_16.iloc[6]
df2.loc[(16,8)]['FR'][:] = fr_1_16.iloc[7]
df2.loc[(16,9)]['FR'][:] = fr_1_16.iloc[8]
df2.loc[(16,10)]['FR'][:] = fr_1_16.iloc[9]

In [65]:
df2.loc[(32,1)]['FR'][:] = fr_1_32.iloc[0]
df2.loc[(32,2)]['FR'][:] = fr_1_32.iloc[1]
df2.loc[(32,3)]['FR'][:] = fr_1_32.iloc[2]
df2.loc[(32,4)]['FR'][:] = fr_1_32.iloc[3]
df2.loc[(32,5)]['FR'][:] = fr_1_32.iloc[4]
df2.loc[(32,6)]['FR'][:] = fr_1_32.iloc[5]
df2.loc[(32,7)]['FR'][:] = fr_1_32.iloc[6]
df2.loc[(32,8)]['FR'][:] = fr_1_32.iloc[7]
df2.loc[(32,9)]['FR'][:] = fr_1_32.iloc[8]
df2.loc[(32,10)]['FR'][:] = fr_1_32.iloc[9]

In [66]:
df2.loc[(64,1)]['FR'][:] = fr_1_64.iloc[0]
df2.loc[(64,2)]['FR'][:] = fr_1_64.iloc[1]
df2.loc[(64,3)]['FR'][:] = fr_1_64.iloc[2]
df2.loc[(64,4)]['FR'][:] = fr_1_64.iloc[3]
df2.loc[(64,5)]['FR'][:] = fr_1_64.iloc[4]
df2.loc[(64,6)]['FR'][:] = fr_1_64.iloc[5]
df2.loc[(64,7)]['FR'][:] = fr_1_64.iloc[6]
df2.loc[(64,8)]['FR'][:] = fr_1_64.iloc[7]
df2.loc[(64,9)]['FR'][:] = fr_1_64.iloc[8]
df2.loc[(64,10)]['FR'][:] = fr_1_64.iloc[9]

In [67]:
df2.loc[(128,1)]['FR'][:] = fr_1_128.iloc[0]
df2.loc[(128,2)]['FR'][:] = fr_1_128.iloc[1]
df2.loc[(128,3)]['FR'][:] = fr_1_128.iloc[2]
df2.loc[(128,4)]['FR'][:] = fr_1_128.iloc[3]
df2.loc[(128,5)]['FR'][:] = fr_1_128.iloc[4]
df2.loc[(128,6)]['FR'][:] = fr_1_128.iloc[5]
df2.loc[(128,7)]['FR'][:] = fr_1_128.iloc[6]
df2.loc[(128,8)]['FR'][:] = fr_1_128.iloc[7]
df2.loc[(128,9)]['FR'][:] = fr_1_128.iloc[8]
df2.loc[(128,10)]['FR'][:] = fr_1_128.iloc[9]

In [68]:
df2.loc[(256,1)]['FR'][:] = fr_1_256.iloc[0]
df2.loc[(256,2)]['FR'][:] = fr_1_256.iloc[1]
df2.loc[(256,3)]['FR'][:] = fr_1_256.iloc[2]
df2.loc[(256,4)]['FR'][:] = fr_1_256.iloc[3]
df2.loc[(256,5)]['FR'][:] = fr_1_256.iloc[4]
df2.loc[(256,6)]['FR'][:] = fr_1_256.iloc[5]
df2.loc[(256,7)]['FR'][:] = fr_1_256.iloc[6]
df2.loc[(256,8)]['FR'][:] = fr_1_256.iloc[7]
df2.loc[(256,9)]['FR'][:] = fr_1_256.iloc[8]
df2.loc[(256,10)]['FR'][:] = fr_1_256.iloc[9]

Now ccs

In [69]:
df2.loc[(2,1)]['CC_5'][:] = cc_1_2_5.iloc[0]
df2.loc[(2,2)]['CC_5'][:] = cc_1_2_5.iloc[1]
df2.loc[(2,3)]['CC_5'][:] = cc_1_2_5.iloc[2]
df2.loc[(2,4)]['CC_5'][:] = cc_1_2_5.iloc[3]
df2.loc[(2,5)]['CC_5'][:] = cc_1_2_5.iloc[4]
df2.loc[(2,6)]['CC_5'][:] = cc_1_2_5.iloc[5]
df2.loc[(2,7)]['CC_5'][:] = cc_1_2_5.iloc[6]
df2.loc[(2,8)]['CC_5'][:] = cc_1_2_5.iloc[7]
df2.loc[(2,9)]['CC_5'][:] = cc_1_2_5.iloc[8]
df2.loc[(2,10)]['CC_5'][:] = cc_1_2_5.iloc[9]

In [70]:
df2.loc[(4,1)]['CC_5'][:] = cc_1_4_5.iloc[0]
df2.loc[(4,2)]['CC_5'][:] = cc_1_4_5.iloc[1]
df2.loc[(4,3)]['CC_5'][:] = cc_1_4_5.iloc[2]
df2.loc[(4,4)]['CC_5'][:] = cc_1_4_5.iloc[3]
df2.loc[(4,5)]['CC_5'][:] = cc_1_4_5.iloc[4]
df2.loc[(4,6)]['CC_5'][:] = cc_1_4_5.iloc[5]
df2.loc[(4,7)]['CC_5'][:] = cc_1_4_5.iloc[6]
df2.loc[(4,8)]['CC_5'][:] = cc_1_4_5.iloc[7]
df2.loc[(4,9)]['CC_5'][:] = cc_1_4_5.iloc[8]
df2.loc[(4,10)]['CC_5'][:] = cc_1_4_5.iloc[9]

In [71]:
df2.loc[(8,1)]['CC_5'][:] = cc_1_8_5.iloc[0]
df2.loc[(8,2)]['CC_5'][:] = cc_1_8_5.iloc[1]
df2.loc[(8,3)]['CC_5'][:] = cc_1_8_5.iloc[2]
df2.loc[(8,4)]['CC_5'][:] = cc_1_8_5.iloc[3]
df2.loc[(8,5)]['CC_5'][:] = cc_1_8_5.iloc[4]
df2.loc[(8,6)]['CC_5'][:] = cc_1_8_5.iloc[5]
df2.loc[(8,7)]['CC_5'][:] = cc_1_8_5.iloc[6]
df2.loc[(8,8)]['CC_5'][:] = cc_1_8_5.iloc[7]
df2.loc[(8,9)]['CC_5'][:] = cc_1_8_5.iloc[8]
df2.loc[(8,10)]['CC_5'][:] = cc_1_8_5.iloc[9]

In [72]:
df2.loc[(16,1)]['CC_5'][:] = cc_1_16_5.iloc[0]
df2.loc[(16,2)]['CC_5'][:] = cc_1_16_5.iloc[1]
df2.loc[(16,3)]['CC_5'][:] = cc_1_16_5.iloc[2]
df2.loc[(16,4)]['CC_5'][:] = cc_1_16_5.iloc[3]
df2.loc[(16,5)]['CC_5'][:] = cc_1_16_5.iloc[4]
df2.loc[(16,6)]['CC_5'][:] = cc_1_16_5.iloc[5]
df2.loc[(16,7)]['CC_5'][:] = cc_1_16_5.iloc[6]
df2.loc[(16,8)]['CC_5'][:] = cc_1_16_5.iloc[7]
df2.loc[(16,9)]['CC_5'][:] = cc_1_16_5.iloc[8]
df2.loc[(16,10)]['CC_5'][:] = cc_1_16_5.iloc[9]

In [73]:
df2.loc[(32,1)]['CC_5'][:] = cc_1_32_5.iloc[0]
df2.loc[(32,2)]['CC_5'][:] = cc_1_32_5.iloc[1]
df2.loc[(32,3)]['CC_5'][:] = cc_1_32_5.iloc[2]
df2.loc[(32,4)]['CC_5'][:] = cc_1_32_5.iloc[3]
df2.loc[(32,5)]['CC_5'][:] = cc_1_32_5.iloc[4]
df2.loc[(32,6)]['CC_5'][:] = cc_1_32_5.iloc[5]
df2.loc[(32,7)]['CC_5'][:] = cc_1_32_5.iloc[6]
df2.loc[(32,8)]['CC_5'][:] = cc_1_32_5.iloc[7]
df2.loc[(32,9)]['CC_5'][:] = cc_1_32_5.iloc[8]
df2.loc[(32,10)]['CC_5'][:] = cc_1_32_5.iloc[9]

In [74]:
df2.loc[(64,1)]['CC_5'][:] = cc_1_64_5.iloc[0]
df2.loc[(64,2)]['CC_5'][:] = cc_1_64_5.iloc[1]
df2.loc[(64,3)]['CC_5'][:] = cc_1_64_5.iloc[2]
df2.loc[(64,4)]['CC_5'][:] = cc_1_64_5.iloc[3]
df2.loc[(64,5)]['CC_5'][:] = cc_1_64_5.iloc[4]
df2.loc[(64,6)]['CC_5'][:] = cc_1_64_5.iloc[5]
df2.loc[(64,7)]['CC_5'][:] = cc_1_64_5.iloc[6]
df2.loc[(64,8)]['CC_5'][:] = cc_1_64_5.iloc[7]
df2.loc[(64,9)]['CC_5'][:] = cc_1_64_5.iloc[8]
df2.loc[(64,10)]['CC_5'][:] = cc_1_64_5.iloc[9]

In [75]:
df2.loc[(128,1)]['CC_5'][:] = cc_1_128_5.iloc[0]
df2.loc[(128,2)]['CC_5'][:] = cc_1_128_5.iloc[1]
df2.loc[(128,3)]['CC_5'][:] = cc_1_128_5.iloc[2]
df2.loc[(128,4)]['CC_5'][:] = cc_1_128_5.iloc[3]
df2.loc[(128,5)]['CC_5'][:] = cc_1_128_5.iloc[4]
df2.loc[(128,6)]['CC_5'][:] = cc_1_128_5.iloc[5]
df2.loc[(128,7)]['CC_5'][:] = cc_1_128_5.iloc[6]
df2.loc[(128,8)]['CC_5'][:] = cc_1_128_5.iloc[7]
df2.loc[(128,9)]['CC_5'][:] = cc_1_128_5.iloc[8]
df2.loc[(128,10)]['CC_5'][:] = cc_1_128_5.iloc[9]

In [76]:
df2.loc[(256,1)]['CC_5'][:] = cc_1_256_5.iloc[0]
df2.loc[(256,2)]['CC_5'][:] = cc_1_256_5.iloc[1]
df2.loc[(256,3)]['CC_5'][:] = cc_1_256_5.iloc[2]
df2.loc[(256,4)]['CC_5'][:] = cc_1_256_5.iloc[3]
df2.loc[(256,5)]['CC_5'][:] = cc_1_256_5.iloc[4]
df2.loc[(256,6)]['CC_5'][:] = cc_1_256_5.iloc[5]
df2.loc[(256,7)]['CC_5'][:] = cc_1_256_5.iloc[6]
df2.loc[(256,8)]['CC_5'][:] = cc_1_256_5.iloc[7]
df2.loc[(256,9)]['CC_5'][:] = cc_1_256_5.iloc[8]
df2.loc[(256,10)]['CC_5'][:] = cc_1_256_5.iloc[9]

In [77]:
df2.loc[(2,1)]['CC_10'][:] = cc_1_2_10.iloc[0]
df2.loc[(2,2)]['CC_10'][:] = cc_1_2_10.iloc[1]
df2.loc[(2,3)]['CC_10'][:] = cc_1_2_10.iloc[2]
df2.loc[(2,4)]['CC_10'][:] = cc_1_2_10.iloc[3]
df2.loc[(2,5)]['CC_10'][:] = cc_1_2_10.iloc[4]
df2.loc[(2,6)]['CC_10'][:] = cc_1_2_10.iloc[5]
df2.loc[(2,7)]['CC_10'][:] = cc_1_2_10.iloc[6]
df2.loc[(2,8)]['CC_10'][:] = cc_1_2_10.iloc[7]
df2.loc[(2,9)]['CC_10'][:] = cc_1_2_10.iloc[8]
df2.loc[(2,10)]['CC_10'][:] = cc_1_2_10.iloc[9]

In [78]:
df2.loc[(4,1)]['CC_10'][:] = cc_1_4_10.iloc[0]
df2.loc[(4,2)]['CC_10'][:] = cc_1_4_10.iloc[1]
df2.loc[(4,3)]['CC_10'][:] = cc_1_4_10.iloc[2]
df2.loc[(4,4)]['CC_10'][:] = cc_1_4_10.iloc[3]
df2.loc[(4,5)]['CC_10'][:] = cc_1_4_10.iloc[4]
df2.loc[(4,6)]['CC_10'][:] = cc_1_4_10.iloc[5]
df2.loc[(4,7)]['CC_10'][:] = cc_1_4_10.iloc[6]
df2.loc[(4,8)]['CC_10'][:] = cc_1_4_10.iloc[7]
df2.loc[(4,9)]['CC_10'][:] = cc_1_4_10.iloc[8]
df2.loc[(4,10)]['CC_10'][:] = cc_1_4_10.iloc[9]

In [79]:
df2.loc[(8,1)]['CC_10'][:] = cc_1_8_10.iloc[0]
df2.loc[(8,2)]['CC_10'][:] = cc_1_8_10.iloc[1]
df2.loc[(8,3)]['CC_10'][:] = cc_1_8_10.iloc[2]
df2.loc[(8,4)]['CC_10'][:] = cc_1_8_10.iloc[3]
df2.loc[(8,5)]['CC_10'][:] = cc_1_8_10.iloc[4]
df2.loc[(8,6)]['CC_10'][:] = cc_1_8_10.iloc[5]
df2.loc[(8,7)]['CC_10'][:] = cc_1_8_10.iloc[6]
df2.loc[(8,8)]['CC_10'][:] = cc_1_8_10.iloc[7]
df2.loc[(8,9)]['CC_10'][:] = cc_1_8_10.iloc[8]
df2.loc[(8,10)]['CC_10'][:] = cc_1_8_10.iloc[9]

In [80]:
df2.loc[(16,1)]['CC_10'][:] = cc_1_16_10.iloc[0]
df2.loc[(16,2)]['CC_10'][:] = cc_1_16_10.iloc[1]
df2.loc[(16,3)]['CC_10'][:] = cc_1_16_10.iloc[2]
df2.loc[(16,4)]['CC_10'][:] = cc_1_16_10.iloc[3]
df2.loc[(16,5)]['CC_10'][:] = cc_1_16_10.iloc[4]
df2.loc[(16,6)]['CC_10'][:] = cc_1_16_10.iloc[5]
df2.loc[(16,7)]['CC_10'][:] = cc_1_16_10.iloc[6]
df2.loc[(16,8)]['CC_10'][:] = cc_1_16_10.iloc[7]
df2.loc[(16,9)]['CC_10'][:] = cc_1_16_10.iloc[8]
df2.loc[(16,10)]['CC_10'][:] = cc_1_16_10.iloc[9]

In [81]:
df2.loc[(32,1)]['CC_10'][:] = cc_1_32_10.iloc[0]
df2.loc[(32,2)]['CC_10'][:] = cc_1_32_10.iloc[1]
df2.loc[(32,3)]['CC_10'][:] = cc_1_32_10.iloc[2]
df2.loc[(32,4)]['CC_10'][:] = cc_1_32_10.iloc[3]
df2.loc[(32,5)]['CC_10'][:] = cc_1_32_10.iloc[4]
df2.loc[(32,6)]['CC_10'][:] = cc_1_32_10.iloc[5]
df2.loc[(32,7)]['CC_10'][:] = cc_1_32_10.iloc[6]
df2.loc[(32,8)]['CC_10'][:] = cc_1_32_10.iloc[7]
df2.loc[(32,9)]['CC_10'][:] = cc_1_32_10.iloc[8]
df2.loc[(32,10)]['CC_10'][:] = cc_1_32_10.iloc[9]

In [82]:
df2.loc[(64,1)]['CC_10'][:] = cc_1_64_10.iloc[0]
df2.loc[(64,2)]['CC_10'][:] = cc_1_64_10.iloc[1]
df2.loc[(64,3)]['CC_10'][:] = cc_1_64_10.iloc[2]
df2.loc[(64,4)]['CC_10'][:] = cc_1_64_10.iloc[3]
df2.loc[(64,5)]['CC_10'][:] = cc_1_64_10.iloc[4]
df2.loc[(64,6)]['CC_10'][:] = cc_1_64_10.iloc[5]
df2.loc[(64,7)]['CC_10'][:] = cc_1_64_10.iloc[6]
df2.loc[(64,8)]['CC_10'][:] = cc_1_64_10.iloc[7]
df2.loc[(64,9)]['CC_10'][:] = cc_1_64_10.iloc[8]
df2.loc[(64,10)]['CC_10'][:] = cc_1_64_10.iloc[9]

In [83]:
df2.loc[(128,1)]['CC_10'][:] = cc_1_128_10.iloc[0]
df2.loc[(128,2)]['CC_10'][:] = cc_1_128_10.iloc[1]
df2.loc[(128,3)]['CC_10'][:] = cc_1_128_10.iloc[2]
df2.loc[(128,4)]['CC_10'][:] = cc_1_128_10.iloc[3]
df2.loc[(128,5)]['CC_10'][:] = cc_1_128_10.iloc[4]
df2.loc[(128,6)]['CC_10'][:] = cc_1_128_10.iloc[5]
df2.loc[(128,7)]['CC_10'][:] = cc_1_128_10.iloc[6]
df2.loc[(128,8)]['CC_10'][:] = cc_1_128_10.iloc[7]
df2.loc[(128,9)]['CC_10'][:] = cc_1_128_10.iloc[8]
df2.loc[(128,10)]['CC_10'][:] = cc_1_128_10.iloc[9]

In [84]:
df2.loc[(256,1)]['CC_10'][:] = cc_1_256_10.iloc[0]
df2.loc[(256,2)]['CC_10'][:] = cc_1_256_10.iloc[1]
df2.loc[(256,3)]['CC_10'][:] = cc_1_256_10.iloc[2]
df2.loc[(256,4)]['CC_10'][:] = cc_1_256_10.iloc[3]
df2.loc[(256,5)]['CC_10'][:] = cc_1_256_10.iloc[4]
df2.loc[(256,6)]['CC_10'][:] = cc_1_256_10.iloc[5]
df2.loc[(256,7)]['CC_10'][:] = cc_1_256_10.iloc[6]
df2.loc[(256,8)]['CC_10'][:] = cc_1_256_10.iloc[7]
df2.loc[(256,9)]['CC_10'][:] = cc_1_256_10.iloc[8]
df2.loc[(256,10)]['CC_10'][:] = cc_1_256_10.iloc[9]

In [85]:
df2.loc[(2,1)]['CC_20'][:] = cc_1_2_20.iloc[0]
df2.loc[(2,2)]['CC_20'][:] = cc_1_2_20.iloc[1]
df2.loc[(2,3)]['CC_20'][:] = cc_1_2_20.iloc[2]
df2.loc[(2,4)]['CC_20'][:] = cc_1_2_20.iloc[3]
df2.loc[(2,5)]['CC_20'][:] = cc_1_2_20.iloc[4]
df2.loc[(2,6)]['CC_20'][:] = cc_1_2_20.iloc[5]
df2.loc[(2,7)]['CC_20'][:] = cc_1_2_20.iloc[6]
df2.loc[(2,8)]['CC_20'][:] = cc_1_2_20.iloc[7]
df2.loc[(2,9)]['CC_20'][:] = cc_1_2_20.iloc[8]
df2.loc[(2,10)]['CC_20'][:] = cc_1_2_20.iloc[9]

In [86]:
df2.loc[(4,1)]['CC_20'][:] = cc_1_4_20.iloc[0]
df2.loc[(4,2)]['CC_20'][:] = cc_1_4_20.iloc[1]
df2.loc[(4,3)]['CC_20'][:] = cc_1_4_20.iloc[2]
df2.loc[(4,4)]['CC_20'][:] = cc_1_4_20.iloc[3]
df2.loc[(4,5)]['CC_20'][:] = cc_1_4_20.iloc[4]
df2.loc[(4,6)]['CC_20'][:] = cc_1_4_20.iloc[5]
df2.loc[(4,7)]['CC_20'][:] = cc_1_4_20.iloc[6]
df2.loc[(4,8)]['CC_20'][:] = cc_1_4_20.iloc[7]
df2.loc[(4,9)]['CC_20'][:] = cc_1_4_20.iloc[8]
df2.loc[(4,10)]['CC_20'][:] = cc_1_4_20.iloc[9]

In [87]:
df2.loc[(8,1)]['CC_20'][:] = cc_1_8_20.iloc[0]
df2.loc[(8,2)]['CC_20'][:] = cc_1_8_20.iloc[1]
df2.loc[(8,3)]['CC_20'][:] = cc_1_8_20.iloc[2]
df2.loc[(8,4)]['CC_20'][:] = cc_1_8_20.iloc[3]
df2.loc[(8,5)]['CC_20'][:] = cc_1_8_20.iloc[4]
df2.loc[(8,6)]['CC_20'][:] = cc_1_8_20.iloc[5]
df2.loc[(8,7)]['CC_20'][:] = cc_1_8_20.iloc[6]
df2.loc[(8,8)]['CC_20'][:] = cc_1_8_20.iloc[7]
df2.loc[(8,9)]['CC_20'][:] = cc_1_8_20.iloc[8]
df2.loc[(8,10)]['CC_20'][:] = cc_1_8_20.iloc[9]

In [88]:
df2.loc[(16,1)]['CC_20'][:] = cc_1_16_20.iloc[0]
df2.loc[(16,2)]['CC_20'][:] = cc_1_16_20.iloc[1]
df2.loc[(16,3)]['CC_20'][:] = cc_1_16_20.iloc[2]
df2.loc[(16,4)]['CC_20'][:] = cc_1_16_20.iloc[3]
df2.loc[(16,5)]['CC_20'][:] = cc_1_16_20.iloc[4]
df2.loc[(16,6)]['CC_20'][:] = cc_1_16_20.iloc[5]
df2.loc[(16,7)]['CC_20'][:] = cc_1_16_20.iloc[6]
df2.loc[(16,8)]['CC_20'][:] = cc_1_16_20.iloc[7]
df2.loc[(16,9)]['CC_20'][:] = cc_1_16_20.iloc[8]
df2.loc[(16,10)]['CC_20'][:] = cc_1_16_20.iloc[9]

In [89]:
df2.loc[(32,1)]['CC_20'][:] = cc_1_32_20.iloc[0]
df2.loc[(32,2)]['CC_20'][:] = cc_1_32_20.iloc[1]
df2.loc[(32,3)]['CC_20'][:] = cc_1_32_20.iloc[2]
df2.loc[(32,4)]['CC_20'][:] = cc_1_32_20.iloc[3]
df2.loc[(32,5)]['CC_20'][:] = cc_1_32_20.iloc[4]
df2.loc[(32,6)]['CC_20'][:] = cc_1_32_20.iloc[5]
df2.loc[(32,7)]['CC_20'][:] = cc_1_32_20.iloc[6]
df2.loc[(32,8)]['CC_20'][:] = cc_1_32_20.iloc[7]
df2.loc[(32,9)]['CC_20'][:] = cc_1_32_20.iloc[8]
df2.loc[(32,10)]['CC_20'][:] = cc_1_32_20.iloc[9]

In [90]:
df2.loc[(64,1)]['CC_20'][:] = cc_1_64_20.iloc[0]
df2.loc[(64,2)]['CC_20'][:] = cc_1_64_20.iloc[1]
df2.loc[(64,3)]['CC_20'][:] = cc_1_64_20.iloc[2]
df2.loc[(64,4)]['CC_20'][:] = cc_1_64_20.iloc[3]
df2.loc[(64,5)]['CC_20'][:] = cc_1_64_20.iloc[4]
df2.loc[(64,6)]['CC_20'][:] = cc_1_64_20.iloc[5]
df2.loc[(64,7)]['CC_20'][:] = cc_1_64_20.iloc[6]
df2.loc[(64,8)]['CC_20'][:] = cc_1_64_20.iloc[7]
df2.loc[(64,9)]['CC_20'][:] = cc_1_64_20.iloc[8]
df2.loc[(64,10)]['CC_20'][:] = cc_1_64_20.iloc[9]

In [91]:
df2.loc[(128,1)]['CC_20'][:] = cc_1_128_20.iloc[0]
df2.loc[(128,2)]['CC_20'][:] = cc_1_128_20.iloc[1]
df2.loc[(128,3)]['CC_20'][:] = cc_1_128_20.iloc[2]
df2.loc[(128,4)]['CC_20'][:] = cc_1_128_20.iloc[3]
df2.loc[(128,5)]['CC_20'][:] = cc_1_128_20.iloc[4]
df2.loc[(128,6)]['CC_20'][:] = cc_1_128_20.iloc[5]
df2.loc[(128,7)]['CC_20'][:] = cc_1_128_20.iloc[6]
df2.loc[(128,8)]['CC_20'][:] = cc_1_128_20.iloc[7]
df2.loc[(128,9)]['CC_20'][:] = cc_1_128_20.iloc[8]
df2.loc[(128,10)]['CC_20'][:] = cc_1_128_20.iloc[9]

In [92]:
df2.loc[(256,1)]['CC_20'][:] = cc_1_256_20.iloc[0]
df2.loc[(256,2)]['CC_20'][:] = cc_1_256_20.iloc[1]
df2.loc[(256,3)]['CC_20'][:] = cc_1_256_20.iloc[2]
df2.loc[(256,4)]['CC_20'][:] = cc_1_256_20.iloc[3]
df2.loc[(256,5)]['CC_20'][:] = cc_1_256_20.iloc[4]
df2.loc[(256,6)]['CC_20'][:] = cc_1_256_20.iloc[5]
df2.loc[(256,7)]['CC_20'][:] = cc_1_256_20.iloc[6]
df2.loc[(256,8)]['CC_20'][:] = cc_1_256_20.iloc[7]
df2.loc[(256,9)]['CC_20'][:] = cc_1_256_20.iloc[8]
df2.loc[(256,10)]['CC_20'][:] = cc_1_256_20.iloc[9]

In [93]:
df2.loc[(2,1)]['CC_50'][:] = cc_1_2_50.iloc[0]
df2.loc[(2,2)]['CC_50'][:] = cc_1_2_50.iloc[1]
df2.loc[(2,3)]['CC_50'][:] = cc_1_2_50.iloc[2]
df2.loc[(2,4)]['CC_50'][:] = cc_1_2_50.iloc[3]
df2.loc[(2,5)]['CC_50'][:] = cc_1_2_50.iloc[4]
df2.loc[(2,6)]['CC_50'][:] = cc_1_2_50.iloc[5]
df2.loc[(2,7)]['CC_50'][:] = cc_1_2_50.iloc[6]
df2.loc[(2,8)]['CC_50'][:] = cc_1_2_50.iloc[7]
df2.loc[(2,9)]['CC_50'][:] = cc_1_2_50.iloc[8]
df2.loc[(2,10)]['CC_50'][:] = cc_1_2_50.iloc[9]

In [94]:
df2.loc[(4,1)]['CC_50'][:] = cc_1_4_50.iloc[0]
df2.loc[(4,2)]['CC_50'][:] = cc_1_4_50.iloc[1]
df2.loc[(4,3)]['CC_50'][:] = cc_1_4_50.iloc[2]
df2.loc[(4,4)]['CC_50'][:] = cc_1_4_50.iloc[3]
df2.loc[(4,5)]['CC_50'][:] = cc_1_4_50.iloc[4]
df2.loc[(4,6)]['CC_50'][:] = cc_1_4_50.iloc[5]
df2.loc[(4,7)]['CC_50'][:] = cc_1_4_50.iloc[6]
df2.loc[(4,8)]['CC_50'][:] = cc_1_4_50.iloc[7]
df2.loc[(4,9)]['CC_50'][:] = cc_1_4_50.iloc[8]
df2.loc[(4,10)]['CC_50'][:] = cc_1_4_50.iloc[9]

In [95]:
df2.loc[(8,1)]['CC_50'][:] = cc_1_8_50.iloc[0]
df2.loc[(8,2)]['CC_50'][:] = cc_1_8_50.iloc[1]
df2.loc[(8,3)]['CC_50'][:] = cc_1_8_50.iloc[2]
df2.loc[(8,4)]['CC_50'][:] = cc_1_8_50.iloc[3]
df2.loc[(8,5)]['CC_50'][:] = cc_1_8_50.iloc[4]
df2.loc[(8,6)]['CC_50'][:] = cc_1_8_50.iloc[5]
df2.loc[(8,7)]['CC_50'][:] = cc_1_8_50.iloc[6]
df2.loc[(8,8)]['CC_50'][:] = cc_1_8_50.iloc[7]
df2.loc[(8,9)]['CC_50'][:] = cc_1_8_50.iloc[8]
df2.loc[(8,10)]['CC_50'][:] = cc_1_8_50.iloc[9]

In [96]:
df2.loc[(16,1)]['CC_50'][:] = cc_1_16_50.iloc[0]
df2.loc[(16,2)]['CC_50'][:] = cc_1_16_50.iloc[1]
df2.loc[(16,3)]['CC_50'][:] = cc_1_16_50.iloc[2]
df2.loc[(16,4)]['CC_50'][:] = cc_1_16_50.iloc[3]
df2.loc[(16,5)]['CC_50'][:] = cc_1_16_50.iloc[4]
df2.loc[(16,6)]['CC_50'][:] = cc_1_16_50.iloc[5]
df2.loc[(16,7)]['CC_50'][:] = cc_1_16_50.iloc[6]
df2.loc[(16,8)]['CC_50'][:] = cc_1_16_50.iloc[7]
df2.loc[(16,9)]['CC_50'][:] = cc_1_16_50.iloc[8]
df2.loc[(16,10)]['CC_50'][:] = cc_1_16_50.iloc[9]

In [97]:
df2.loc[(32,1)]['CC_50'][:] = cc_1_32_50.iloc[0]
df2.loc[(32,2)]['CC_50'][:] = cc_1_32_50.iloc[1]
df2.loc[(32,3)]['CC_50'][:] = cc_1_32_50.iloc[2]
df2.loc[(32,4)]['CC_50'][:] = cc_1_32_50.iloc[3]
df2.loc[(32,5)]['CC_50'][:] = cc_1_32_50.iloc[4]
df2.loc[(32,6)]['CC_50'][:] = cc_1_32_50.iloc[5]
df2.loc[(32,7)]['CC_50'][:] = cc_1_32_50.iloc[6]
df2.loc[(32,8)]['CC_50'][:] = cc_1_32_50.iloc[7]
df2.loc[(32,9)]['CC_50'][:] = cc_1_32_50.iloc[8]
df2.loc[(32,10)]['CC_50'][:] = cc_1_32_50.iloc[9]

In [98]:
df2.loc[(64,1)]['CC_50'][:] = cc_1_64_50.iloc[0]
df2.loc[(64,2)]['CC_50'][:] = cc_1_64_50.iloc[1]
df2.loc[(64,3)]['CC_50'][:] = cc_1_64_50.iloc[2]
df2.loc[(64,4)]['CC_50'][:] = cc_1_64_50.iloc[3]
df2.loc[(64,5)]['CC_50'][:] = cc_1_64_50.iloc[4]
df2.loc[(64,6)]['CC_50'][:] = cc_1_64_50.iloc[5]
df2.loc[(64,7)]['CC_50'][:] = cc_1_64_50.iloc[6]
df2.loc[(64,8)]['CC_50'][:] = cc_1_64_50.iloc[7]
df2.loc[(64,9)]['CC_50'][:] = cc_1_64_50.iloc[8]
df2.loc[(64,10)]['CC_50'][:] = cc_1_64_50.iloc[9]

In [99]:
df2.loc[(128,1)]['CC_50'][:] = cc_1_128_50.iloc[0]
df2.loc[(128,2)]['CC_50'][:] = cc_1_128_50.iloc[1]
df2.loc[(128,3)]['CC_50'][:] = cc_1_128_50.iloc[2]
df2.loc[(128,4)]['CC_50'][:] = cc_1_128_50.iloc[3]
df2.loc[(128,5)]['CC_50'][:] = cc_1_128_50.iloc[4]
df2.loc[(128,6)]['CC_50'][:] = cc_1_128_50.iloc[5]
df2.loc[(128,7)]['CC_50'][:] = cc_1_128_50.iloc[6]
df2.loc[(128,8)]['CC_50'][:] = cc_1_128_50.iloc[7]
df2.loc[(128,9)]['CC_50'][:] = cc_1_128_50.iloc[8]
df2.loc[(128,10)]['CC_50'][:] = cc_1_128_50.iloc[9]

In [100]:
df2.loc[(256,1)]['CC_50'][:] = cc_1_256_50.iloc[0]
df2.loc[(256,2)]['CC_50'][:] = cc_1_256_50.iloc[1]
df2.loc[(256,3)]['CC_50'][:] = cc_1_256_50.iloc[2]
df2.loc[(256,4)]['CC_50'][:] = cc_1_256_50.iloc[3]
df2.loc[(256,5)]['CC_50'][:] = cc_1_256_50.iloc[4]
df2.loc[(256,6)]['CC_50'][:] = cc_1_256_50.iloc[5]
df2.loc[(256,7)]['CC_50'][:] = cc_1_256_50.iloc[6]
df2.loc[(256,8)]['CC_50'][:] = cc_1_256_50.iloc[7]
df2.loc[(256,9)]['CC_50'][:] = cc_1_256_50.iloc[8]
df2.loc[(256,10)]['CC_50'][:] = cc_1_256_50.iloc[9]

Save the dataframe

In [101]:
df2

CV           FR      CC_5     CC_10     CC_20     CC_50
2   1  0     0.176262  0.0325 1/ms  0.022812  0.038135  0.066177 -0.026174
       1     0.177784  0.0324 1/ms -0.004292 -0.019779 -0.005685  0.065774
       2     0.182037  0.0324 1/ms -0.032714 -0.085546 -0.099031 -0.026174
       3     0.179802  0.0321 1/ms  0.026304 -0.002609 -0.018727 -0.036038
       4     0.181406  0.0325 1/ms -0.049676 -0.072938 -0.059902  0.002498
...               ...          ...       ...       ...       ...       ...
256 10 9995   0.16278  0.0323 1/ms  0.010483 -0.019193 -0.039419 -0.024806
       9996  0.188127  0.0319 1/ms  0.015987  0.017055   0.08348  0.099277
       9997  0.195896  0.0322 1/ms  0.031881  0.015545  0.007319  0.061448
       9998  0.174652  0.0323 1/ms  0.001987  0.014029  0.019055  0.050453
       9999  0.172298  0.0316 1/ms -0.015566 -0.047835 -0.072414 -0.057284

[800000 rows x 6 columns]

In [102]:
df2.to_pickle(r'{}\semi_cont_brunel_df.pkl'.format('C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data'))